In [1]:
import sys
import os
ROOT_DIR = os.path.abspath("/MoStress/")
sys.path.append(ROOT_DIR)

In [2]:
import json
from os.path import join

import numpy as np
import pandas as pd
from kerasbeats import prep_time_series
from utils.preprocessingCheckpoint import getPreprocessingCheckpoint
from models.architectures.NBEATS.NBEATSModel import NBEATSModel
from tensorflow import keras
import tensorflow as tf


In [3]:
trainData, validationData = getPreprocessingCheckpoint()

In [4]:
dataset = {
    "features": trainData["features"],
    "targets": trainData["targets"],
    "weights": trainData["weights"],
}

In [5]:
inputTest = dataset["features"][0]

inputTest

array([[-0.04906158,  0.32236075,  2.31629804, -0.70362568,  0.35170601],
       [ 0.00304426, -0.309945  ,  2.28104558, -0.45453641,  0.43153586],
       [ 0.62631692,  0.45954869,  2.19640463, -0.31583771,  0.497078  ],
       ...,
       [-0.00608964, -0.49451708,  1.34191815, -2.03553343, -0.74628732],
       [ 0.23747035,  0.34576267,  1.38512626, -1.23622251, -0.75530661],
       [-2.68725794,  1.26433206,  1.45325614, -2.09487152, -0.75049123]])

In [6]:
inputTest.shape

(420, 5)

In [7]:
signalTest = inputTest[ : , 0 ]
signalTest.shape

(420,)

In [8]:
X, y = prep_time_series(signalTest, lookback = 7, horizon = 1)

In [9]:
X.shape, y.shape

((413, 7), (413, 1))

In [10]:
nBeats = NBEATSModel()

In [11]:
import tensorflow.keras.backend as K

In [ ]:
tf.config.run_functions_eagerly(True)

In [12]:
nBeats.fit(X, y, epochs = 10, verbose = True, use_multiprocessing=True)

2022-10-01 16:06:02.822344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
1/1 [==============================] - 1s 911ms/step - loss: 0.9635 - mae: 0.9635 - mape: 1266.0277
Epoch 2/10
1/1 [==============================] - 1s 552ms/step - loss: 11.5869 - mae: 11.5869 - mape: 19763.1699
Epoch 3/10
1/1 [==============================] - 1s 553ms/step - loss: 10.8454 - mae: 10.8454 - mape: 14831.5879
Epoch 4/10
1/1 [==============================] - 1s 552ms/step - loss: 3.3813 - mae: 3.3813 - mape: 4971.3872
Epoch 5/10
1/1 [==============================] - 1s 554ms/step - loss: 1.8437 - mae: 1.8437 - mape: 2865.6924
Epoch 6/10
1/1 [==============================] - 1s 550ms/step - loss: 1.1038 - mae: 1.1038 - mape: 1929.1271
Epoch 7/10
1/1 [==============================] - 1s 570ms/step - loss: 0.7007 - mae: 0.7007 - mape: 960.4575
Epoch 8/10
1/1 [==============================] - 1s 568ms/step - loss: 1.1656 - mae: 1.1656 - mape: 1534.2986
Epoch 9/10
1/1 [==============================] - 1s 544ms/step - loss: 1.2026 - mae: 1.2026 - mape: 1621.0

In [17]:
# nBeats.model_layer._residual_collection

In [18]:
class CustomCallback(keras.callbacks.Callback):
    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))
        print("\n")
        print(f"Esse é o batch: {batch} e esse é o output: {logs['outputs']}")
        forecasts, residuals, backcastHist = self.model.get_layer("nbeats").output
        print("\n")
        print(f"forecast: {forecasts} \n type {type(forecasts)} \n value? {K.eval(forecasts)} ")
        print("\n")
        print(f"residual: {residuals} \n type {type(residuals[0])} ")
        print("\n")
        print(f"backcastHist: {backcastHist} ")
        

In [19]:
predictions = nBeats.predict(X, callbacks=[CustomCallback()])

...Predicting: end of batch 0; got log keys: ['outputs']


Esse é o batch: 0 e esse é o output: [[ 0.16310538]
 [ 0.13463801]
 [ 0.1815725 ]
 [ 0.6857682 ]
 [ 5.921306  ]
 [ 0.16603762]
 [ 0.4821756 ]
 [ 0.117966  ]
 [ 0.11101583]
 [ 0.22188789]
 [ 0.33769125]
 [ 0.21121284]
 [ 0.01802515]
 [ 0.48957124]
 [-0.19298632]
 [ 0.10318866]
 [ 0.20535341]
 [ 0.32067832]
 [ 0.4380926 ]
 [-1.8292035 ]
 [ 0.29281524]
 [ 0.42288792]
 [ 0.1760519 ]
 [-0.04439881]
 [ 0.04788899]
 [ 0.49549618]
 [ 1.0540838 ]
 [-0.05648454]
 [ 1.0165308 ]
 [-0.07184599]
 [ 0.12086914]
 [-0.5122214 ]]




/Users/I546644/Personal/Unisinos/Codes/MoStress/mostress-dev2-env/lib/python3.9/site-packages/tensorflow/python/data/ops/structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


AttributeError: 'KerasTensor' object has no attribute 'numpy'

In [ ]:
backcastHist = nBeats.backcastHist.predict(X, callbacks=[CustomCallback()])

In [ ]:
residuals = nBeats.residuals.predict(X, callbacks=[CustomCallback()])

In [ ]:
X

In [ ]:
reverseX = K.reverse(X, axes=0)

In [ ]:
reverseX = np.array(reverseX)

In [ ]:
reverseX

In [ ]:
backcastHist[0]

In [ ]:
backcastHist[0] - X

In [ ]:
backcastHist[0] - reverseX

In [ ]:
residuals[0]

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
plot_model(nBeats.residuals, show_layer_names=True)

In [ ]:
nBeats.model

In [ ]:
print(tf.__version__)

In [ ]:
from tensorflow import keras
import tensorflow.keras.backend as K

In [ ]:
X

In [ ]:
resid = K.reverse(X, axes=0)
resid

In [ ]:
teste = keras.layers.Subtract()([resid, X])

In [ ]:
teste

In [ ]:
timeInputLayers = []
nbeatsLayers = []
for i in range(inputTest.shape[1]):
    time_input = keras.layers.Input(shape = (7, ))
    nbeats = NBeatsModel(model_type = 'generic', lookback = 7, horizon = 1).build_layer()
    timeInputLayers.append(time_input)
    nbeatsLayers.append(nbeats.model_layer(time_input))

In [ ]:
concat = keras.layers.Concatenate()(nbeatsLayers)
output = keras.layers.Dense(3, activation="softmax")(concat)
model = keras.Model(inputs = timeInputLayers, outputs = output)

In [ ]:
model.compile(loss = 'sparse_categorical_crossentropy')

In [ ]:
x = [ convert_to_tensor(inputTest[:, i]) for i in range(inputTest.shape[1]) ]
y = [
    np.array(
        [
            np.int64(randint(0, 2))
            for _ in range(inputTest.shape[0])
        ]
    )
    for _ in range(inputTest.shape[1])
]

In [ ]:
x = stack(x)
y = convert_to_tensor(stack(y))

In [ ]:
model.fit(x, y, verbose = True)